In [2]:
%pip install scikit-learn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 31.6 kB/s eta 0:00:0000:0100:08
  Using cached scipy-1.11.4-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (36.4 MB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.2/302.2 KB 23.1 kB/s eta 0:00:0000:0100:01
Note: you may need to restart the kernel to use updated packages.


In [26]:
import pandas as pd
import streamlit as st
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import BaggingRegressor
from sklearn.ensemble import StackingRegressor
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeRegressor

# Charger le dataset
# Assurez-vous d'ajuster le chemin du fichier CSV en conséquence
df = pd.read_csv("/home/FIL_ROUGE/Historical/data/raw/openmeteo/open-meteo-2000-2023.csv", parse_dates=["time"])

df['time'] = pd.to_datetime(df['time']).dt.tz_localize(None)  # Retirer le fuseau horaire

df.fillna(0, inplace=True)

# Extraire des caractéristiques pertinentes de la colonne "time"
df['hour'] = df['time'].dt.hour
df['day_of_week'] = df['time'].dt.dayofweek
df['month'] = df['time'].dt.month

# Séparer les features et la target
X = df.drop(columns=['rain (mm)', 'time'])
y = df['rain (mm)']

# Diviser le dataset en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [27]:
# Initialiser les modèles de base
models = [
    RandomForestRegressor(random_state=42),
    GradientBoostingRegressor(random_state=42),
    DecisionTreeRegressor(random_state=42),
]

# Créer le modèle de Bagging
bagging_model = BaggingRegressor(base_estimator=None, n_estimators=10, random_state=42)

# Entraîner le modèle de Bagging
bagging_model.fit(X_train, y_train)

# Faire des prédictions sur l'ensemble de test
y_pred_bagging = bagging_model.predict(X_test)

# Calculer l'erreur quadratique moyenne (RMSE)
rmse_bagging = mean_squared_error(y_test, y_pred_bagging, squared=False)
print(f'RMSE (Bagging): {rmse_bagging}')


/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_base.py:156: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(


RMSE (Bagging): 0.00279667712129172


In [24]:
%pip install joblib

Note: you may need to restart the kernel to use updated packages.


In [28]:
import joblib

# Sauvegarder le modèle dans un fichier
joblib.dump(bagging_model, 'bagging_model.joblib')


['bagging_model.joblib']

In [19]:
import numpy as np

# Vous pouvez ajuster cela en fonction des caractéristiques réelles utilisées dans votre modèle
feature_columns = X_train.columns

# Générer des valeurs aléatoires pour chaque caractéristique
random_values = np.random.rand(1, len(feature_columns))

# Créer un dataframe avec ces valeurs aléatoires
random_input = pd.DataFrame(random_values, columns=feature_columns)

# Faire la prédiction avec le modèle entraîné
predicted_rain = bagging_model.predict(random_input)

# print("Valeurs d'entrée aléatoires:")
print(random_input)
print("\n\n")
print("****************************************************************************")
print("\nPrédiction de la pluie (mm):", predicted_rain[0])
print("\n\n")

print("****************************************************************************")


   temperature_2m (°C)  relativehumidity_2m (%)  dewpoint_2m (°C)  \
0             0.327817                 0.069682          0.033535   

   apparent_temperature (°C)  precipitation (mm)  snowfall (cm)  \
0                   0.476305            0.190481       0.731238   

   snow_depth (m)  weathercode (wmo code)  pressure_msl (hPa)  \
0        0.424842                0.463721            0.848468   

   surface_pressure (hPa)  ...  direct_normal_irradiance (W/m²)  \
0                0.317523  ...                         0.776348   

   terrestrial_radiation (W/m²)  shortwave_radiation_instant (W/m²)  \
0                      0.847609                            0.591905   

   direct_radiation_instant (W/m²)  diffuse_radiation_instant (W/m²)  \
0                         0.629875                          0.928606   

   direct_normal_irradiance_instant (W/m²)  \
0                                 0.750275   

   terrestrial_radiation_instant (W/m²)      hour  day_of_week     month  
0   

In [21]:
# 1. Prédiction de la quantité de pluie par mois
df['predicted_rain'] = bagging_model.predict(X)  # Utiliser le modèle pour prédire la pluie
df['month'] = df['time'].dt.month

# Agréger par mois
monthly_rain = df.groupby(['month'])['predicted_rain'].sum()

# Afficher la quantité de pluie par mois
print("Quantité de pluie par mois :")
print(monthly_rain)

# 2. Prédiction de la quantité de pluie par an
df['year'] = df['time'].dt.year

# Agréger par an
annual_rain = df.groupby(['year'])['predicted_rain'].sum()

# Afficher la quantité de pluie par an
print("\nQuantité de pluie par an :")
print(annual_rain)

# 3. Système d'alerte
seuil_secheresse = 100  # Seuil pour déclencher une alerte de sécheresse
seuil_inondation = 500  # Seuil pour déclencher une alerte d'inondation

# Déclencher des alertes en fonction des seuils
alert_secheresse = annual_rain.min() < seuil_secheresse
alert_inondation = annual_rain.max() > seuil_inondation

# Afficher les alertes
print("\nAlerte de sécheresse :", alert_secheresse)
print("Alerte d'inondation :", alert_inondation)


Quantité de pluie par mois :
month
1     135.13
2     321.79
3     206.28
4      86.03
5     104.17
6      51.40
7      21.50
8      48.02
9     225.08
10    272.97
11    356.63
12    156.63
Name: predicted_rain, dtype: float64

Quantité de pluie par an :
year
2000     16.22
2001     19.30
2002     83.40
2003    110.33
2004     65.39
2005     70.91
2006    177.74
2007     46.16
2008    132.09
2009     96.62
2010    122.56
2011     94.53
2012     39.00
2013     52.67
2014    235.82
2015     61.09
2016     90.92
2017     40.14
2018     92.58
2019     51.70
2020     59.52
2021     76.94
2022     75.32
2023     74.68
Name: predicted_rain, dtype: float64

Alerte de sécheresse : True
Alerte d'inondation : False


In [22]:
%pip install streamlit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 229.6 kB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.4/196.4 KB 439.8 kB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.6/294.6 KB 493.2 kB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.1/82.1 KB 347.1 kB/s eta 0:00:0000:0100:01
  Using cached tenacity-8.2.3-py3-none-any.whl (24 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 185.9 kB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 165.8 kB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 240.6/240.6 KB 468.3 kB/s eta 0:00:00a 0:00:01
  Using cached click-8.1.7-py3-none-any.whl (97 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 996.9/996.9 KB 125.0 kB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.0/38.0 MB 227.4 kB/s eta 0:00:0000:0100:05
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.8/55.8 KB 177.1 kB/s eta 0:00:0

In [29]:
X_train.columns

Index(['temperature_2m (°C)', 'relativehumidity_2m (%)', 'dewpoint_2m (°C)',
       'apparent_temperature (°C)', 'precipitation (mm)', 'snowfall (cm)',
       'snow_depth (m)', 'weathercode (wmo code)', 'pressure_msl (hPa)',
       'surface_pressure (hPa)', 'cloudcover (%)', 'cloudcover_low (%)',
       'cloudcover_mid (%)', 'cloudcover_high (%)',
       'et0_fao_evapotranspiration (mm)', 'vapor_pressure_deficit (kPa)',
       'windspeed_10m (m/s)', 'windspeed_100m (m/s)', 'winddirection_10m (°)',
       'winddirection_100m (°)', 'windgusts_10m (m/s)',
       'soil_temperature_0_to_7cm (°C)', 'soil_temperature_7_to_28cm (°C)',
       'soil_temperature_28_to_100cm (°C)',
       'soil_temperature_100_to_255cm (°C)', 'soil_moisture_0_to_7cm (m³/m³)',
       'soil_moisture_7_to_28cm (m³/m³)', 'soil_moisture_28_to_100cm (m³/m³)',
       'soil_moisture_100_to_255cm (m³/m³)', 'shortwave_radiation (W/m²)',
       'direct_radiation (W/m²)', 'diffuse_radiation (W/m²)',
       'direct_normal_irra

In [23]:
import streamlit as st
import pandas as pd
import numpy as np

# Charger les données (remplacez le chemin par le vôtre)
df = pd.read_csv("/home/FIL_ROUGE/Historical/data/raw/openmeteo/open-meteo-2000-2023.csv", parse_dates=["time"])

# Simulation de prédictions pour les besoins de l'exemple
df['predicted_rain'] = np.random.rand(len(df)) * 10  # Remplacez ceci par vos prédictions réelles

# Calculer la quantité de pluie par mois
df['month'] = df['time'].dt.month
monthly_rain = df.groupby(['month'])['predicted_rain'].sum()

# Calculer la quantité de pluie par an
df['year'] = df['time'].dt.year
annual_rain = df.groupby(['year'])['predicted_rain'].sum()

# Seuils pour les alertes
seuil_secheresse = 100
seuil_inondation = 500

# Déclencher des alertes en fonction des seuils
alert_secheresse = annual_rain.min() < seuil_secheresse
alert_inondation = annual_rain.max() > seuil_inondation

# Interface Streamlit
st.title("Prédictions de Pluie et Alertes Météo")

# Affichage de la quantité de pluie par mois
st.subheader("Quantité de pluie par mois")
st.bar_chart(monthly_rain)

# Affichage de la quantité de pluie par an
st.subheader("Quantité de pluie par an")
st.bar_chart(annual_rain)

# Affichage des alertes météo
st.subheader("Alertes Météo")
st.write(f"Alerte de sécheresse : {alert_secheresse}")
st.write(f"Alerte d'inondation : {alert_inondation}")


2024-01-17 15:16:06.823 
  command:

    streamlit run /root/.local/lib/python3.10/site-packages/ipykernel_launcher.py [ARGUMENTS]
